Note: All code in script is sourced from various internet sources, from mentor Tomasz Popiel and from Alan Chalk at Sabre Insurance.

## Distance Matrix

Contents:

 - Start_: import modules, set directories, load data ('02_df_all.pickle') 
 
 -  Sample non-scaled data
 
 - Scaling (numerical and histogram variables)
 
 - Distance matrix components (Numerical, Categorical, Histogram)
 
 -  Combining into final distance matrix
 
 -  Save  
 ('03_distancematrix_20%_exclude_target_weight511.hkl')  
 ('03_distancematrix_20%_exclude_target_weight151.hkl')  
 ('03_distancematrix_20%_exclude_target_weight115.hkl')

Notes: 

- Add the weight for experiment
- Exclude target for target analysis 

### Start_:

Import modules

In [0]:
import os
import pickle
import gc
import sys

import pandas as pd
import numpy as np

from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import jensenshannon

from sklearn.preprocessing import normalize

Set directories

In [0]:
print(os.getcwd())

dirRawData = "../RawData/"
dirPData = "../PData/"
dirPOutput = "../POutput/"

/Users/hujuiting/Desktop/GROUP PROJECT/PCode


Load data

In [0]:
fname = dirPData + '02_df_all.pickle' 
with open(fname, 'rb') as f:
    dict_ = pickle.load(f)

df_all = dict_['df_all']
del fname
df_all.shape

(65196, 643)

### Sample non-scaled data

In [0]:
df_all_2 = df_all.sample(frac=0.2, random_state=123)

In [0]:
df_all_2.head()

,tq_db11_mean,tq_db12_mean,tq_db13_mean,tq_db14_mean,tq_dt1_mean,tq_dt2_mean,tq_dt3_mean,tq_dt4_mean,tq_db15_mean,tq_db16_mean,...,a_v7_8__97_431,a_v7_8__97_46,a_v7_8__97_647,a_v7_8__97_770,a_v7_8__97_875,a_v7_8__97_909,a_v7_8__99_37,a_v7_8__99_995,a_v7_8__99_996,a_v7_8_other
12437,15.9672,7.3934,1.4918,0.0164,0.0984,0.0328,0.0,0.0164,0.0984,0.0984,...,0,0,0,0,0,0,0,0,0,0
32372,16.3125,4.4375,1.1875,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,0
26155,20.0000,6.8462,1.1538,0.0769,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,0
50441,16.5000,5.0000,1.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,0
45622,19.1250,9.8750,1.7500,0.0000,0.1250,0.0000,0.0,0.0000,0.3750,0.3750,...,0,0,0,0,0,0,0,0,0,0


### Scaling (numerical and histogram variables)

Numerical variables : normalize 

In [0]:
df_scaled = normalize(df_all[dict_['vars_numeric']])
df_scaled = pd.DataFrame(df_scaled, columns=df_all[dict_['vars_numeric']].columns)

In [0]:
for col in dict_['vars_numeric']:
    df_all[col] = df_scaled[col]

Histogram variables : divide by 100 so they sum to 1

In [0]:
for col in dict_['vars_hist']:
    df_all[col] = df_all[col]/100

In [0]:
del df_scaled
gc.collect()

40

### Distance matrix components

Create distance matrices for numerical, categorial and histogram varaibles separately, then later add together to get final distance matrix

#### Set weights for numerical, categorical and histgoram

In [0]:
weight_num = 1
weight_cat = 5
weight_his = 1

#### Downsample data

In [0]:
# randomly sample 20% of the dataset
df_all = df_all.sample(frac=0.2, random_state=123)

In [0]:
# Drop ID as is not useful for clustering
vars_notToUse = ['id',
 'tq_dt1_mean',
 'tq_dt2_mean',
 'tq_dt3_mean',
 'tq_dt4_mean',
 'tq_dt1_std',
 'tq_dt2_std',
 'tq_dt3_std',
 'tq_dt4_std']

vars_toUse = [var for var in df_all.columns if var not in vars_notToUse]

In [0]:
df_ds = df_all[vars_toUse]

In [0]:
df_ds.shape

(13039, 634)

### Numerical Variables

Using the 'pdist' function and distance metric "cityblock"   

Considering different types of numerical varaibles separately (mean, std, mode, NMiss, tq_v3, a_v6, a_v9, a_v10, a_v11, a_v12)

In [0]:
vars_num_prefix_1 = ('_mean','_Mean')

In [0]:
vars_num_prefix_2 = ('_std','_StdDev')

In [0]:
vars_num_prefix_3 = ('_Mode', '_NMiss')

In [0]:
vars_num_prefix_4 = ['tq_v3','a_v6', 'a_v9', 'a_v10', 'a_v11', 'a_v12']

In [0]:
### exclude the target variables 

vars_num_separate = []

vars_num_separate.append([col for col in dict_['vars_numeric'] if col.endswith(vars_num_prefix_1) and not col.startswith('tq_dt1') and not col.startswith ('tq_dt2') and not col.startswith('tq_dt3') and not col.startswith ('tq_dt4')])

vars_num_separate.append([col for col in dict_['vars_numeric'] if col.endswith(vars_num_prefix_2) and not col.startswith('tq_dt1') and not col.startswith ('tq_dt2') and not col.startswith('tq_dt3') and not col.startswith ('tq_dt4')])

for prefix in vars_num_prefix_3:
    vars_num_separate.append([col for col in dict_['vars_numeric'] if col.endswith(prefix)])

vars_num_separate.append(vars_num_prefix_4)

In [0]:
# we have a total of 5 types of numerical variables
len(vars_num_separate)

5

In [0]:
#vars_num_separate

Combine the 5 types of numerical varaibles distance matrices into one distance matrix 'num_dist_mat" and add weights. Have chosen weight 1 so no weight for now. This can be adjusted

In [0]:
num_dist_mat = None
weights = [1, 1, 1, 1, 1]

for idx, number in enumerate(vars_num_separate):
    number_as_matrix = df_ds[number].values
    each_distance_matrix = pdist(number_as_matrix, metric='cityblock')
    del number_as_matrix
    gc.collect()
    print(each_distance_matrix)
    #print(sys.getsizeof(each_distance_matrix)/1e9)
    if num_dist_mat is None:
        num_dist_mat = each_distance_matrix * weights[idx]
    else:
        num_dist_mat = np.sum([num_dist_mat, each_distance_matrix * weights[idx]], axis=0)
    del each_distance_matrix
    gc.collect()        

[0.57837191 0.68884797 0.96088635 ... 1.7798353  2.65400556 0.99020731]
[0.29191958 0.31139068 1.23138086 ... 1.03733625 1.63922339 0.60378148]
[2.30695339 2.09398696 1.39625035 ... 2.01703892 2.34294503 0.80124663]
[0.21567934 0.30144201 0.3539212  ... 0.77338483 1.82176689 1.09711334]
[0.30694788 0.16572437 0.20764071 ... 0.31540574 0.41344071 0.15890002]


In [0]:
num_dist_mat

array([3.69987209, 3.56139199, 4.15007947, ..., 5.92300105, 8.87138158,
       3.65124878])

### Categorical Variables

Using the 'pdist' function and distance metric "hamming"   

Considering different types of categorical varaibles separately ('a_v1','a_v2','a_v3','a_v4','a_v5','a_v7_8')

In [0]:
vars_cat_prefix = ('a_v1','a_v2','a_v3','a_v4','a_v5','a_v7_8')

In [0]:
vars_cat_separate = []
for prefix in vars_cat_prefix:
    vars_cat_separate.append([col for col in dict_['vars_cat'] if col.startswith(prefix)])

In [0]:
# we have a total of 6 categories
len(vars_cat_separate)

6

Combine the 6 types of categorical varaibles distance matrices into one distance matrix 'cat_dist_mat" and add weights. Have chosen weight 1 so no weight for now. This can be adjusted

In [0]:
# consider each category in turn, calculating its distance matrix
# weights defined here for each category so that we can add as we go along
cat_dist_mat = None
weights = [1, 1, 1, 1, 1, 1]

for idx, category in enumerate(vars_cat_separate):
    #print(idx)
    category_as_matrix = df_ds[category].values
    each_distance_matrix = pdist(category_as_matrix, metric='hamming')
    print(each_distance_matrix)
    del category_as_matrix
    gc.collect()
    if cat_dist_mat is None:
        cat_dist_mat = each_distance_matrix * weights[idx]
    else:
        cat_dist_mat = np.sum([cat_dist_mat, each_distance_matrix * weights[idx]], axis=0)
    del each_distance_matrix
    gc.collect()

[0. 0. 0. ... 0. 0. 0.]
[0.         0.66666667 0.         ... 0.66666667 0.         0.66666667]
[0.66666667 0.         0.66666667 ... 0.         0.         0.        ]
[0.         0.28571429 0.28571429 ... 0.28571429 0.28571429 0.        ]
[0.4 0.4 0.4 ... 0.4 0.4 0. ]
[0.00692042 0.00692042 0.00692042 ... 0.00692042 0.00692042 0.        ]


In [0]:
cat_dist_mat

array([1.07358708, 1.35930137, 1.35930137, ..., 1.35930137, 0.6926347 ,
       0.66666667])

#### Combine num_dist_mat and cat_dist_mat, weights 1
Combine num_dist_mat and cat_dist_mat to avoid storing both

In [0]:
a, b = num_dist_mat.mean(), cat_dist_mat.mean()
print((a, b))

(4.922322612654607, 1.3525410041234684)


In [0]:
print(num_dist_mat)
print(cat_dist_mat)
dist_mat = np.sum([weight_num * num_dist_mat, 
                   weight_cat * a / b * cat_dist_mat
                  ], axis=0)
print(dist_mat)

[3.69987209 3.56139199 4.15007947 ... 5.92300105 8.87138158 3.65124878]
[1.07358708 1.35930137 1.35930137 ... 1.35930137 0.6926347  0.66666667]
[23.23547935 28.29602051 28.88470799 ... 30.65762957 21.47496047
 15.7822984 ]


In [0]:
#mean values are the same 
(weight_cat * a / b * cat_dist_mat).mean()

24.611613063272934

In [0]:
(weight_num * num_dist_mat).mean()

4.922322612654607

In [0]:
del num_dist_mat, cat_dist_mat
gc.collect()

120

### Histogram Variables

Using the 'pdist' function and distance metric "jensenshannon"   

Considering different histogram varaibles separately

In [0]:
vars_hist_prefix = ('tq_db24', 'tq_db25', 'tq_db27', 'tq_db28', 'tq_db29', 'tq_db30','tq_db31',
                           'tq_v4', 'tq_v5', 'tq_db32', 'tq_db33', 'tq_db34', 
                           'tq_da12','tq_da13', 'tq_da14',
                           'tq_db35', 'tq_db36','tq_db37', 'tq_db38',
                           'tq_db39', 'tq_db40', 'tq_db41', 'tq_db42',
                           'tq_db43', 'tq_db45', 'tq_db46',
                           'tq_db47', 'tq_db48', 'tq_db49', 'tq_db50')

In [0]:
vars_hist_separate = []
for prefix in vars_hist_prefix:
    vars_hist_separate.append([col for col in dict_['vars_hist'] if col.startswith(prefix)])

In [0]:
#vars_hist_separate

In [0]:
# we have a total of 30 histograms
len(vars_hist_separate)

30

Combine the 30 histogram varaibles distance matrices into one distance matrix 'hist_dist_mat" and add weights. Have chosen weight 1 so no weight for now. This can be adjusted

In [0]:
hist_dist_mat = None
weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

for idx, histogram in enumerate(vars_hist_separate):
    #print(idx)
    histogram_as_matrix = df_ds[histogram].values
    each_distance_matrix = pdist(histogram_as_matrix, metric='jensenshannon')
    print(each_distance_matrix)
    del histogram_as_matrix
    gc.collect()
    if hist_dist_mat is None:
        hist_dist_mat = each_distance_matrix * weights[idx]
    else:
        hist_dist_mat = np.sum([hist_dist_mat, each_distance_matrix * weights[idx]], axis=0)
    del each_distance_matrix
    gc.collect()

[0.32110426 0.02630462 0.7266391  ... 0.07540514 0.26874401 0.19678598]
[0.         0.         0.         ... 0.05273159 0.04166534 0.01325817]
[0.17577947 0.10663291 0.42821022 ... 0.19713337 0.2288473  0.04856455]
[0.11384739 0.07656987 0.36037072 ... 0.04116571 0.03323081 0.07431246]
[0.30412519 0.29689756 0.44052665 ... 0.2478376  0.24869964 0.0985484 ]
[0.65673232 0.64414187 0.78745939 ... 0.36314955 0.37851526 0.19271047]
[0.2189326  0.25761886 0.54726255 ... 0.17260587 0.14243383 0.04191737]
[0.30858726 0.30550598 0.72655756 ... 0.7069239  0.72178872 0.06520844]
[0.07468223 0.10663291 0.07468223 ... 0.         0.01861985 0.01861985]
[0.05780056 0.15466943 0.48424546 ... 0.30233582 0.26649434 0.0939827 ]
[0.10758877 0.06914959 0.10758877 ... 0.17298864 0.13224733 0.06930745]
[0.10758877 0.06914959 0.10758877 ... 0.15765851 0.10814969 0.08639448]
[0.49549307 0.5092181  0.70161271 ... 0.64971755 0.56336008 0.35952711]
[0.25468087 0.07946667 0.74024851 ... 0.29865737 0.01508642 0.28

In [0]:
hist_dist_mat                                                                                                                                                                                                                                                                                                                                                                                                       

array([ 7.11066234,  4.95312266, 11.57415155, ...,  6.56310686,
        6.30195136,  3.46067731])

### Combining into final distance matrix
Combining the numerical, categorical and histogram distance matrices into one distance matrix D_3 (apply weights above)

In [0]:
c = hist_dist_mat.mean()
print((a, b, c))

(4.922322612654607, 1.3525410041234684, 8.020125426935385)


In [0]:
# rescale so so mean values are the same so each matrix has an equal weighting 
# and no matrix is negligible (these weights can be adjusted)
dist_mat = np.sum([dist_mat, weight_his * a / c * hist_dist_mat], axis=0)

In [0]:
# mean values are the same
(weight_his * a / c * hist_dist_mat).mean()

4.922322612654603

In [0]:
del hist_dist_mat
gc.collect()

100

In [0]:
D_3 = dist_mat
del dist_mat
gc.collect()

20

### Save

In [0]:
import hickle as hkl 

dict_['df_all_2'] = df_all_2  # 20% of non scaled dataset (with variables not to use) 

dict_['df_all'] = df_all  # 20% of scaling dataset (with variables not to use)

dict_['D_3'] = D_3 # a condensed distance matrix

#dict_['D_3_square'] = D_3_square # symmetric square matrix

#fname = dirPData + '03_distancematrix_20%_exclude_target_weight511.hkl'
fname = dirPData + '03_distancematrix_20%_exclude_target_weight151.hkl'
#fname = dirPData + '03_distancematrix_20%_exclude_target_weight115.hkl'

with open(fname, 'w') as f:
    hkl.dump(dict_, f)

del dict_, fname, df_all

/Users/hujuiting/anaconda3/lib/python3.7/site-packages/hickle/hickle.py:403: SerializedWarning: <class 'pandas.core.frame.DataFrame'> type not understood, data have been serialized
  SerializedWarning)
